In [1]:
using Pkg
Pkg.activate("../Dashboard/env")

  Activating project at `c:\Users\Vlad\Projects\Dashboard\env`


In [3]:
list = ["Makie", "GLMakie", "PyMNE", "JLD2", "TopoPlots", "StatsBase", "Pipe", "ColorSchemes", "Colors", "LinearAlgebra", "AbstractPlotting"]
 
for i in list
    import Pkg; Pkg.add(i);
    print(i, '\n')
end 

In [2]:
using Makie
using GLMakie
using PyMNE
using JLD2 # loading data
using TopoPlots
using StatsBase # mean/std
using Pipe
using ColorSchemes
using Colors
using LinearAlgebra


In [13]:
Makie.inline!(false)

false

In [3]:
begin # load  one single-subject dataset 
	#p = "../../../../store/users/ehinger/projects/unfoldjl_dev/dev/UnfoldMakie/artifact/sub-002_ses-N170_task-N170_eeg.set"
	p = "../EEG_plots/data/sub-002_ses-N170_task-N170_eeg.set"
    raw = PyMNE.io.read_raw_eeglab(p,preload=true)
	#p_all = "../../../../store/users/ehinger/projects/unfoldjl_dev/data/erpcore-N170.jld2"
    p_all = "../EEG_plots/data/erpcore-N170.jld2"
	presaved_data = load(p_all)
	dat_e = presaved_data["data_e_all"].* 1e6
	evt_e = presaved_data["df_e_all"]
	mon = PyMNE.channels.make_standard_montage("standard_1020")
	raw.set_channel_types(Dict("HEOG_left"=>"eog","HEOG_right"=>"eog","VEOG_lower"=>"eog"))
	raw.set_montage(mon,match_case=false)
	pos = PyMNE.channels.make_eeg_layout(get_info(raw)).pos
	pos = [Point2f(pos[k,1],pos[k,2]) for k in 1:size(pos,1)]
end;

┌ Warning: type DataFrames.DataFrame does not exist in workspace; reconstructing
└ @ JLD2 C:\Users\Vlad\.julia\packages\JLD2\1YVED\src\data\reconstructing_datatypes.jl:495
┌ Warning: type DataFrames.Index does not exist in workspace; reconstructing
└ @ JLD2 C:\Users\Vlad\.julia\packages\JLD2\1YVED\src\data\reconstructing_datatypes.jl:495


In [18]:
using Pkg; Pkg.Pkg.add(PackageSpec(name="Makie", version="0.19.0"))#update("Makie")

   Resolving package versions...

Pkg.Resolve.ResolverError: Unsatisfiable requirements detected for package [38;5;2mMakie [ee78f7c6][39m:
 [38;5;2mMakie [ee78f7c6][39m log:
 ├─possible versions are: [38;5;2m0.9.0-0.19.0[39m or uninstalled
 ├─restricted to versions [38;5;2m0.19.0[39m by an explicit requirement, leaving only versions [38;5;2m0.19.0[39m
 └─restricted by compatibility requirements with [38;5;3mTopoPlots [2bdbdf9c][39m to versions: [38;5;2m0.17.8-0.18.4[39m — no versions left
   └─[38;5;3mTopoPlots [2bdbdf9c][39m log:
     ├─possible versions are: [38;5;3m0.1.0-0.1.2[39m or uninstalled
     └─restricted to versions [38;5;3m*[39m by an explicit requirement, leaving only versions [38;5;3m0.1.0-0.1.2[39m

# 1. Topoplot with timeslider

- put true timeline
- faster interpolator?

In [141]:
let 
    f = Figure(resolution = (1000, 900))
    xs = range(-0.3, length=size(dat_e, 2), step=1 ./ 128) 
    sg = SliderGrid(f[2, 1],
        (label="time", range=xs, format = "{:.3f} ms", startvalue = 0),
    )
    time = sg.sliders[1].value
    str = lift(t -> "[$(round(t, digits = 3)) ms]", time)
    topo_slice = lift((t, data) -> mean(data[1:30, indexin(t, xs), :], dims=2)[:,1], time, dat_e)
    topo_axis = Axis(f[1, 1], aspect = DataAspect(), title = "Interactive topoplot")
    topo = eeg_topoplot!(topo_axis, topo_slice, # averaging all trial of 30 participants on Xth msec
        raw.ch_names[1:30]; 
        positions=pos, # produced  automatically from ch_names
        #interpolation=DelaunayMesh(),
        enlarge=1.2,
        markersize = 10,
        lablesize = 10,
        extrapolation=GeomExtrapolation(enlarge=1.3, geometry=Circle),
        label_text=true) # aspect ratio, correlation of height and width
    
    # decrement/increment slider with left/right keys
    on(events(f).keyboardbutton) do btn
        if btn.action in (Keyboard.press, Keyboard.repeat)
            if btn.key == Keyboard.left
                set_close_to!(sg.sliders[1], time[] - 1)
            elseif btn.key == Keyboard.right
                set_close_to!(sg.sliders[1], time[] + 1)
            end
        end
    end

    text!(topo_axis, 1, 1, text = str,  align = (:center, :center))
    xlims!(-0.2, 1.2)
    ylims!(-0.2, 1.2)
    hidedecorations!(topo_axis)
    hidespines!(topo_axis) 
    f
# animation
#= 
framerate = 1
timestamps = [-0.3, 0.0828125, 0.1609375, 0.2390625]
record(f, "animations/topoplot.gif", timestamps;
        framerate = framerate) do z
    sg.sliders[1].value[] = z
    time[] = z
end =#
end

- increases the size of labels
- change the position of labels

# animation

In [149]:
f = Figure(resolution = (1000, 900))
xs = range(-0.3, length=size(dat_e, 2), step=1 ./ 128) 
sg = SliderGrid(f[2, 1],
    (label="time", range=xs, format = "{:.3f} ms", startvalue = 0),
)
time = sg.sliders[1].value
str = lift(t -> "[$(round(t, digits = 3)) ms]", time)
topo_slice = lift((t, data) -> mean(data[1:30, indexin(t, xs), :], dims=2)[:,1], time, dat_e)
topo_axis = Axis(f[1, 1], aspect = DataAspect(), title = "Interactive topoplot")
topo = eeg_topoplot!(topo_axis, topo_slice, # averaging all trial of 30 participants on Xth msec
    raw.ch_names[1:30]; 
    positions=pos, # produced  automatically from ch_names
    #interpolation=DelaunayMesh(),
    enlarge=1.2,
    markersize = 10,
    extrapolation=GeomExtrapolation(enlarge=1.3, geometry=Circle),
    label_text=true) # aspect ratio, correlation of height and width

text!(topo_axis, 1, 1, text = str,  align = (:center, :center))
xlims!(-0.2, 1.2)
ylims!(-0.2, 1.2)
hidedecorations!(topo_axis)
hidespines!(topo_axis) 

# animation
framerate = 1
timestamps = [-0.3, 0.0828125, 0.1609375, 0.2390625]
record(f, "animations/topoplot.gif", timestamps;
        framerate = framerate) do z
    sg.sliders[1].value[] = z
    time[] = z
end

"animations/topoplot.gif"

# 2. Combination of butterflyplot and topoplot

- we want a topoplot in a specific timepoints (at least one)
    - with arrow toword the timepint?
- what kind of interactivity is possible here?
- slider? or froppping list?
- red vline on x axis?
- topoplot on the bottom or on top?

In [19]:
function eegHeadMatrix(positions, center, radius)
    oldCenter = mean(positions)
    oldRadius, _ = findmax(x-> LinearAlgebra.norm(x .- oldCenter), positions)
    radF = radius/oldRadius
    return Makie.Mat4f(radF, 0, 0, 0,
                       0, radF, 0, 0,
                       0, 0, 1, 0,
                       center[1]-oldCenter[1]*radF, center[2]-oldCenter[2]*radF, 0, 1)
end

struct NullInterpolator <: TopoPlots.Interpolator
end

function (ni::NullInterpolator)(
        xrange::LinRange, yrange::LinRange,
        positions::AbstractVector{<: Point{2}}, data::AbstractVector{<:Number})

    return zeros(length(xrange),length(yrange))
end

function posToColor(pos)
    cx = 0.5 - pos[1]
    cy = 0.5 - pos[2]
    rx = cx * 0.7071068 + cy * 0.7071068
    ry = cx * -0.7071068 + cy * 0.7071068
    b = 1.0 - (2*sqrt(cx^2+cy^2))^2
    return RGB(0.5 - rx*1.414, 0.5 - ry*1.414, b)
end

posToColor (generic function with 1 method)

In [46]:
# butterflyplot with interactiv topoplot
let 
    f = Figure(backgroundcolor = RGBf(0.98, 0.98, 0.98), resolution = (1500, 700))

    # interaction
    xs = range(-0.3, length=size(dat_e, 2), step=1 ./ 128)  
    sg = SliderGrid(f[4, 1:3],
        (label="time", range=xs, format = "{:.3f} ms", startvalue = 0),
    )
    time = sg.sliders[1].value
    str = lift(t -> "$(round(t, digits = 3)) ms", time)
    topo_slice = lift((t, data) -> mean(data[1:30, indexin(t, xs), :], dims=2)[:,1], time, dat_e)

    # butterfly plot
    ax = Axis(f[2:3, 1:3], xlabel = "Time [s]", ylabel = "Voltage amplitude [µV]")
    N = 1:length(pos) #1:4 
    hidespines!(ax, :t, :r) 
    GLMakie.xlims!(-0.3, 1.2)
    hlines!(0, color = :gray, linewidth = 1)
    vlines!(0, color = :gray, linewidth = 1)
    times = range(-0.3, length=size(dat_e,2), step=1 ./ 128)
    specialColors = ColorScheme(vcat(RGB(1,1,1.),[posToColor(pos) for pos in pos[N]]...))

    for i in N
        mean_trial = mean(dat_e[i,:,:], dims=2)[:,1]
        lines!(times, mean_trial, color = specialColors[i])
    end 
    hidedecorations!(ax, label = false, ticks = false, ticklabels = false) 

    # text
    vlines!(time,  color = :red, linewidth = 1)
    

    # topoplot 
    topo_axis = Axis(f[1, 2], width = 178, height = 178, aspect = DataAspect())
	Makie.xlims!(low = -0.2, high = 1.2)
	Makie.ylims!(low = -0.2, high = 1.2)
    topoMatrix = eegHeadMatrix(pos[N], (0.5, 0.5), 0.5)

    topo = eeg_topoplot!(topo_axis, topo_slice, # averaging all trial of 30 participants on Xth msec
        raw.ch_names[1:30]; 
        positions=pos, # produced  automatically from ch_names
        #interpolation=DelaunayMesh(),
        enlarge=1,
        extrapolation=GeomExtrapolation(enlarge=1.0, geometry=Circle),
        label_text=false)
     
    hidedecorations!(current_axis())
    hidespines!(current_axis())

    #a3 = Axis()
    Label(f[1, 2], str,
        textsize = 36,
        #fontsize = 26,
        font = :bold,
        padding = (40, 500, 0, 0),
        halign = :right)

    #text!(time, 0, labelsize = 50, text = str,  align = (:center, :center))   
    hidedecorations!(current_axis())
    hidespines!(current_axis())
    f

    # animation
    framerate = 1
    timestamps = [ 0.0046875,    0.1609375,    0.3171875,    0.4734375,    0.6296875,    0.7859375,    0.9421875]

    record(f, "animations/butterfly_topoplot.gif", timestamps;
            framerate = framerate) do z
        sg.sliders[1].value = z
        time[] = z
    end
end

"animations/butterfly_topoplot.gif"

# 3. Combination of butterflyplot and topoplot series

- butterflyplot
- topoplot series underneath 
- you can change number of topoplots and their range

LoadError: LoadError: UndefVarError: @pipe not defined
in expression starting at c:\Users\Vlad\Projects\Dashboard\prototype.ipynb:2

# 4. Buterfly plot with interactvie mode 

- two modes: ordinary and interactive
- in interactvie mode: i click the timeserises and its related postion on topoplot shine red

In [51]:
# classic butterflyplot
let    
    f = Figure(backgroundcolor = RGBf(0.98, 0.98, 0.98), resolution = (1500, 700))
    ax = Axis(f[1:2,1], xlabel = "Time [s]", ylabel = "Voltage amplitude [µV]")
    
    N = 1:length(pos) #1:4 
    hidespines!(ax, :t, :r) 
    GLMakie.xlims!(-0.3, 1.2)
    hlines!(0, color = :gray, linewidth = 1)
    vlines!(0, color = :gray, linewidth = 1)
    times = range(-0.3, length=size(dat_e,2), step=1 ./ 128)
    specialColors = ColorScheme(vcat(RGB(1,1,1.),[posToColor(pos) for pos in pos[N]]...))

    for i in N
        mean_trial = mean(dat_e[i,:,:],dims=2)[:,1]
        lines!(times, mean_trial, color = specialColors[i])
    end 
    hidedecorations!(ax, label = false, ticks = false, ticklabels = false) 

    topo_axis = Axis(f[1:2, 2], width = 78, height = 78, aspect = DataAspect())

	Makie.xlims!(low = -0.2, high = 1.2)
	Makie.ylims!(low = -0.2, high = 1.2)
    topoMatrix = eegHeadMatrix(pos[N], (0.5, 0.5), 0.5)
    
    topo = eeg_topoplot!(topo_axis, N, # averaging all trial of 30 participants on Xth msec
        raw.ch_names[1:30]; 
        positions=pos, # produced  automatically from ch_names
        #interpolation=DelaunayMesh(),
        enlarge=1,
        extrapolation=GeomExtrapolation(enlarge=1.0, geometry=Circle),
        label_text=false)
    hidedecorations!(current_axis())
    hidespines!(current_axis())
    f
end